## Data

In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset("json", data_files='alpaca-lora/alpaca_data_cleaned.json')

Found cached dataset json (/home/maziar/.cache/huggingface/datasets/json/default-44bd61d3faf6c7bb/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
data_point = data["train"].shuffle()[0]
instruction = data_point['instruction']
input = data_point['input']
output = data_point['output']

print(f'instruction: {instruction}')
print(f'input: {input}')
print(f'output: {output}')

instruction: Modify the sentence given by removing a word and output the modified sentence.
input: The ship ventured into the dangerous storm.
output: The ship ventured into the storm.


In [4]:
import json
with open('alpaca-lora/templates/alpaca.json') as fp:
    template = json.load(fp)

In [5]:
template

{'description': 'Template used by Alpaca-LoRA.',
 'prompt_input': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n',
 'prompt_no_input': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n',
 'response_split': '### Response:'}

In [6]:
def generate_prompt(template, instruction, input = None, label = None):
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)
    if label:
        res = f"{res}{label}"
    return res

In [7]:
full_prompt = generate_prompt(template, instruction, input, output)
print(full_prompt)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Modify the sentence given by removing a word and output the modified sentence.

### Input:
The ship ventured into the dangerous storm.

### Response:
The ship ventured into the storm.


### Tokenize

In [8]:
from transformers import LlamaTokenizer

In [9]:
tokenizer = LlamaTokenizer.from_pretrained('LLaMA_HF/')

In [10]:
tokenizer.pad_token_id = (0)
tokenizer.padding_side = "left"

In [11]:
tokenizer

LlamaTokenizer(name_or_path='LLaMA_HF/', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'pad_token': '<unk>'}, clean_up_tokenization_spaces=False)

In [12]:
cutoff_len = 256
tokenized_full_prompt = tokenizer(full_prompt, truncation=True, max_length=cutoff_len, padding=False, return_tensors=None)

In [13]:
tokenized_full_prompt

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 2111, 1598, 278, 10541, 2183, 491, 11077, 263, 1734, 322, 1962, 278, 9120, 10541, 29889, 13, 13, 2277, 29937, 10567, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 18215, 14280, 29889, 13, 13, 2277, 29937, 13291, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 14280, 29889], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
add_eos_token = True
if (tokenized_full_prompt["input_ids"][-1] != tokenizer.eos_token_id
    and len(tokenized_full_prompt["input_ids"]) < cutoff_len
    and add_eos_token):
    
    tokenized_full_prompt["input_ids"].append(tokenizer.eos_token_id)
    tokenized_full_prompt["attention_mask"].append(1)

In [15]:
tokenized_full_prompt

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 2111, 1598, 278, 10541, 2183, 491, 11077, 263, 1734, 322, 1962, 278, 9120, 10541, 29889, 13, 13, 2277, 29937, 10567, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 18215, 14280, 29889, 13, 13, 2277, 29937, 13291, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 14280, 29889, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenized_full_prompt["labels"] = tokenized_full_prompt["input_ids"].copy()

In [17]:
tokenized_full_prompt

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 2111, 1598, 278, 10541, 2183, 491, 11077, 263, 1734, 322, 1962, 278, 9120, 10541, 29889, 13, 13, 2277, 29937, 10567, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 18215, 14280, 29889, 13, 13, 2277, 29937, 13291, 29901, 13, 1576, 7751, 9712, 2955, 964, 278, 14280, 29889, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 2993

### Model

In [18]:
from transformers import LlamaForCausalLM

In [19]:
import torch
model = LlamaForCausalLM.from_pretrained('LLaMA_HF/', load_in_8bit=True, torch_dtype=torch.float16, device_map='auto')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/maziar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/maziar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/maziar/miniconda3/envs/pytorch2 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/maziar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/extras/CUPTI/lib64'), PosixPath('/usr/local/cuda-10.1/include'), PosixPath('/usr/local/cuda-10.1/lib64')}
  warn(msg)
/home/maziar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/cuda-10.1/include:/usr/local/cuda-10.1/lib64::/usr/local/cuda/extras/CUPTI/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/maziar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING:

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

In [21]:
from peft import LoraConfig

config = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

In [22]:
from peft import get_peft_model

model = get_peft_model(model, config)

In [23]:
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [24]:
model_input = torch.Tensor([tokenized_full_prompt["input_ids"]]).int()

In [25]:
model_output = model.forward(model_input)

In [26]:
model_output['logits'].shape

torch.Size([1, 86, 32000])

In [27]:
torch.argmax(model_output['logits'], -1)[0]

tensor([  450,   338,   263,   429,   284,   674,   920,  1158,   393,   607,
         2859,   411,   263,  1967,   322,   338,   278,  2472, 29889,    13,
          263,  1824,   393, 16612,  2486,  6089,  2167,   278,  3414, 29889,
           13,  1576, 29905, 29937, 10567,   582,    13,    13,    13,  1598,
          278,  1494,  2400,  2400,   278,   278,  1734,   393, 15270,  1259,
        10541, 10541, 29889,    13,    13,  2277, 29937, 10567, 29901,    13,
         1576, 10541,   471,  2955,   964,   278,  9815, 19922, 29889,    13,
           13,  2277, 29937, 10604, 29901,    13,  1576,  7751,  9712,  2955,
          964,   278, 18215, 29889,    13,     1])

In [28]:
print(tokenizer.decode(torch.argmax(model_output['logits'], -1)[0]))

 The is a exal will how method that whichired with a image and is the information.
 a program that describesately answerstes the task.
The\# Inputru


ify the following below below the the word that replacingting sentence sentence.

### Input:
The sentence wasured into the unknown waters.

### Output:
The ship ventured into the dangerous.

